### Topic 3 - Weighting, Value, and Momentum

### a. Obtaining our data

We will be recreating a version of the DJIA index. However, our index is going to be equally weighted. Our goal is to create a program that will take the value of your portfolio and tell you how much of each company in the DJIA you would need to purchase to create your equally weighted fund.

In [2]:
#pip install XlsxWriter


In [3]:
# For data manipulation
import numpy as np
import pandas as pd

# For obtaining and exporting data
import yfinance as yf
import requests
import xlsxwriter
import ssl

# Additional
import math

In [4]:
# DJIA stock list

djia_url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'

# ignore certificate errors
ssl._create_default_https_context = ssl._create_unverified_context

# Read and print the stock tickers that make up the DJIA
stocks = pd.read_html(djia_url)[1]
display(stocks.head())

,Company,Exchange,Symbol,Industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.41%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.02%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,5.48%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,2.84%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.36%


In [5]:
# Ticker list only

stocks = stocks[['Symbol']]
stocks.sample(10)

,Symbol
5,CAT
27,V
1,AXP
11,GS
20,MSFT
10,DOW
29,WMT
3,AAPL
15,INTC
17,JPM


In [6]:
stock_data = yf.Ticker('WMT')
shares_out = pd.DataFrame(stock_data.get_shares_full(start="2019-01-01", end=None))
shares_out['Year'] = shares_out.index.year
shares_out.rename(columns={0:'BasicShares'}, inplace = True)
shares_out.tail()

,BasicShares,Year
2024-01-17 00:00:00-05:00,2692229888,2024
2024-01-19 00:00:00-05:00,2692229888,2024
2024-01-20 00:00:00-05:00,2692229888,2024
2024-01-25 00:00:00-05:00,2692229888,2024
2024-01-30 00:00:00-05:00,2692229888,2024


In [7]:
price = yf.download('WMT', start = '2019-01-01', auto_adjust = True)[['Close']]
price['Year'] = price.index.year
price.tail()

[*********************100%%**********************]  1 of 1 completed


C:\Users\yagmu\AppData\Local\Temp\ipykernel_14528\3189484850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Year'] = price.index.year


,Close,Year
Date,,
2024-01-24,160.500000,2024
2024-01-25,162.839996,2024
2024-01-26,164.270004,2024
2024-01-29,165.039993,2024
2024-01-30,165.589996,2024


In [8]:
shares_price = pd.merge(shares_out, price, on = 'Year', how = 'inner')
shares_price['marketCap'] = shares_price.eval('Close * BasicShares')
shares_price = shares_price.iloc[-1]
shares_price

BasicShares    2.692230e+09
Year           2.024000e+03
Close          1.655900e+02
marketCap      4.458063e+11
Name: 57545, dtype: float64

In [9]:
# We will now get data for all of DJIA tickers
# We must first setup a dataframe to store our data

my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [10]:
new_row = pd.Series(['WMT', shares_price['Close'], shares_price['marketCap'], 'N/A'], index=my_columns)

# Use pd.concat to add the new row
final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,WMT,165.589996,4.458063e+11,N/A


In [11]:
stocks = stocks.head()
stocks.head(10)

,Symbol
0,MMM
1,AXP
2,AMGN
3,AAPL
4,BA


In [12]:
# We will now loop through our tickers and store the data in a dataframe

final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Symbol']:

    stock_data = yf.Ticker(symbol)
    shares_out = pd.DataFrame(stock_data.get_shares_full(start="2019-01-01", end=None))
    shares_out['Year'] = shares_out.index.year
    shares_out.rename(columns={0:'BasicShares'}, inplace = True)

    price = yf.download(symbol, start = '2019-01-01', auto_adjust = True)[['Close']]
    price['Year'] = price.index.year

    shares_price = pd.merge(shares_out, price, on = 'Year', how = 'inner')
    shares_price['marketCap'] = shares_price.eval('Close * BasicShares')
    shares_price = shares_price.iloc[-1]

    
    new_row = pd.Series([symbol, shares_price['Close'], shares_price['marketCap'], 'N/A'], index=my_columns)

    # Use pd.concat to add the new row
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
    #final_dataframe = final_dataframe.append(pd.Series([symbol, shares_price['Close'], shares_price['marketCap'], 'N/A'], index = my_columns), ignore_index = True)

[*********************100%%**********************]  1 of 1 completed


C:\Users\yagmu\AppData\Local\Temp\ipykernel_14528\2274339762.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Year'] = price.index.year


[*********************100%%**********************]  1 of 1 completed


C:\Users\yagmu\AppData\Local\Temp\ipykernel_14528\2274339762.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Year'] = price.index.year


[*********************100%%**********************]  1 of 1 completed


C:\Users\yagmu\AppData\Local\Temp\ipykernel_14528\2274339762.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Year'] = price.index.year


[*********************100%%**********************]  1 of 1 completed


C:\Users\yagmu\AppData\Local\Temp\ipykernel_14528\2274339762.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Year'] = price.index.year


[*********************100%%**********************]  1 of 1 completed


C:\Users\yagmu\AppData\Local\Temp\ipykernel_14528\2274339762.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Year'] = price.index.year


In [13]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   5 non-null      object 
 1   Price                    5 non-null      float64
 2   Market Capitalization    5 non-null      float64
 3   Number Of Shares to Buy  5 non-null      object 
dtypes: float64(2), object(2)
memory usage: 292.0+ bytes


In [14]:
final_dataframe.head()

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,MMM,95.750000,5.290963e+10,N/A
1,AXP,204.149994,1.476004e+11,N/A
2,AMGN,314.619995,1.683777e+11,N/A
3,AAPL,188.039993,2.907456e+12,N/A
4,BA,200.440002,1.212616e+11,N/A


### b. Calculating number of shares to buy
Remember that we want to create an equally weighted portfolio. This would be very easy if all of our stocks were the same price. Our stocks are not the same price so we must make a few calculations. We will first have to determine how much you are going to invest before deciding how much of each stock you need to purchase. Once we know the total amount to be invested, we can determine how much of each stock will result in an equal weighted portfolio.

In [15]:
portfolio_size = input('How much money are you investing:')

try:
    val = float(portfolio_size)
    print(val)
except:
    print('Please enter an integer value.')
    portfolio_size = input('How much money are you investing:')
    val = float(portfolio_size)
    print(val)

5000.0


In [16]:
# Determine our position size in dollars based on how much we will invest and how many stocks we must purchase.

position_size = val/len(final_dataframe.index)
print(position_size)

1000.0


In [17]:
# How many shares of each stock do we need to purchase to reach $333 invested?
# We will round our number of shares to purchase to the lower full number of shares.

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,MMM,95.750000,5.290963e+10,10
1,AXP,204.149994,1.476004e+11,4
2,AMGN,314.619995,1.683777e+11,3
3,AAPL,188.039993,2.907456e+12,5
4,BA,200.440002,1.212616e+11,4


### c. Format and export this information to Excel
Now that we have this information we do not want to have to run our Python program everytime we want to access it. We will therefore export this data to Excel in a nicely formatted file.

### d. Initalizing our xlsxWriter Object

In [18]:
writer = pd.ExcelWriter('recommended_trades_DJ30.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades DJ30', index = False)

### e. Creating the format we will use for our output file
We are able to format colors, fonts, and symbols.
- String format for tickers
- \\$##.## format for stock prices
- \\$##.### format for market capitalization
- Integer format for the number of shares to purchase

In [19]:
# I would copy and paste this section rather than trying to learn all of the formatting options.

background_color = '#474747'
font_color = '#ffffff'
string_format = writer.book.add_format({'font_color': font_color,'bg_color': background_color,'border': 1})
dollar_format = writer.book.add_format({'num_format':'$0.00','font_color': font_color,'bg_color': background_color,'border': 1})
integer_format = writer.book.add_format({'num_format':'0','font_color': font_color,'bg_color': background_color,'border': 1})

### f. Applying the formats to the columns in our xlsx file

In [20]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades DJ30'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades DJ30'].write(f'{column}1', column_formats[column][0], string_format)

### g. Save our output

In [21]:
# Is the output pretty? No, but you can play around with the formatting options to change that if you would like.

writer.close()

### h. What if we wanted a market cap weighted portfolio?

In [22]:
final_dataframe.head()

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,MMM,95.750000,5.290963e+10,10
1,AXP,204.149994,1.476004e+11,4
2,AMGN,314.619995,1.683777e+11,3
3,AAPL,188.039993,2.907456e+12,5
4,BA,200.440002,1.212616e+11,4


In [23]:
final_dataframe['mcap_weight'] = final_dataframe['Market Capitalization'] / sum(final_dataframe['Market Capitalization'])
final_dataframe.head()

,Ticker,Price,Market Capitalization,Number Of Shares to Buy,mcap_weight
0,MMM,95.750000,5.290963e+10,10,0.015573
1,AXP,204.149994,1.476004e+11,4,0.043442
2,AMGN,314.619995,1.683777e+11,3,0.049558
3,AAPL,188.039993,2.907456e+12,5,0.855737
4,BA,200.440002,1.212616e+11,4,0.035690


In [24]:
# Check that our weights sum to one

final_dataframe['mcap_weight'].sum()

1.0

In [25]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   5 non-null      object 
 1   Price                    5 non-null      float64
 2   Market Capitalization    5 non-null      float64
 3   Number Of Shares to Buy  5 non-null      object 
 4   mcap_weight              5 non-null      float64
dtypes: float64(3), object(2)
memory usage: 332.0+ bytes


In [26]:
# Calculate the new number of shares to buy

final_dataframe['mcap_dollars_to_invest'] = final_dataframe['mcap_weight'] * float(portfolio_size)
final_dataframe['mcap_shares_to_buy'] = final_dataframe['mcap_dollars_to_invest'] / final_dataframe['Price']
final_dataframe['mcap_shares_to_buy'] = final_dataframe['mcap_shares_to_buy'].apply(lambda x: math.floor(x))
final_dataframe.sort_values('Market Capitalization', ascending = False)

,Ticker,Price,Market Capitalization,Number Of Shares to Buy,mcap_weight,mcap_dollars_to_invest,mcap_shares_to_buy
3,AAPL,188.039993,2.907456e+12,5,0.855737,4278.684003,22
2,AMGN,314.619995,1.683777e+11,3,0.049558,247.788803,0
1,AXP,204.149994,1.476004e+11,4,0.043442,217.212486,1
4,BA,200.440002,1.212616e+11,4,0.035690,178.451580,0
0,MMM,95.750000,5.290963e+10,10,0.015573,77.863127,0


## 4. Value portfolio creation
A value portfolio simply tries to buy 'cheap' stocks relative to their intrinsic value. Our goal is going to be to select the 15 best value stocks from the DJIA index. We will then create an equal weighted portfolio from those stocks. This time we will focus on using yfinance and Yahoo Finance to obtain our data.

### a. Obtain a new set of data and include the PE Ratio.

In [35]:
# Additional library needed

from scipy import stats

In [36]:
# url of the source
djia_url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'

# ignore certificate errors
ssl._create_default_https_context = ssl._create_unverified_context

# Read and print the stock tickers that make up the DJIA
djia_tickers = pd.read_html(djia_url)[1]
print('DJIA Tickers')
display(djia_tickers.head())

tickers_list = djia_tickers['Symbol'].tolist()
tickers_list

DJIA Tickers


,Company,Exchange,Symbol,Industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.41%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.02%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,5.48%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,2.84%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.36%


['MMM',
 'AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'CRM',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WBA',
 'WMT']

In [37]:
tickers_data= {} # empty dictionary

for ticker in tickers_list:
    ticker_object = yf.Ticker(ticker)

    #convert info() output from dictionary to dataframe
    temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
    temp.reset_index(inplace=True)
    temp.columns = ["Attribute", "Recent"]

    # add (ticker, dataframe) to main dictionary
    tickers_data[ticker] = temp

tickers_data

HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/MMM?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true

In [ ]:
tickers_data['MMM']['Attribute'].tolist()

KeyError: 'MMM'

In [ ]:
combined_data = pd.concat(tickers_data)
combined_data = combined_data.reset_index()
combined_data.sample(3)

In [ ]:
combined_data.info()

In [ ]:
combined_data.drop(columns = "level_1", inplace = True) # clean up unnecessary column
combined_data.columns = ["Ticker", "Attribute", "Value"] # update column names

combined_data.sample(3)

In [ ]:
combined_data.head(10)

In [ ]:
# Keep only the variables we are interested in here or in our next step

combined_data = combined_data.loc[(combined_data['Attribute'] == 'open') |
                                 (combined_data['Attribute'] == 'forwardPE') |
                                 (combined_data['Attribute'] == 'priceToBook') |
                                 (combined_data['Attribute'] == 'priceToSalesTrailing12Months') |
                                 (combined_data['Attribute'] == 'enterpriseToEbitda')]
combined_data

In [ ]:
# Reformat our data so that we can more easily work with it

combined_data = combined_data.pivot(index = 'Ticker', columns = 'Attribute', values = 'Value').reset_index()

In [ ]:
combined_data.sample(5)

In [ ]:
combined_data.rename(columns={'open': 'Price',
                              'forwardPE': 'Price-to-Earnings Ratio',
                              'priceToBook': 'Price-to-Book Ratio',
                              'priceToSalesTrailing12Months': 'Price-to-Sales Ratio',
                              'enterpriseToEbitda': 'EV-to-EBITDA'}, inplace=True)

combined_data = combined_data.rename_axis(None, axis=1)
combined_data.head()

In [ ]:
# Notice that our data is all formatted as objects

combined_data.info()

In [ ]:
for i in combined_data:
    if i != 'Ticker':
        combined_data[i] = combined_data[i].astype('float')

combined_data.info()

In [ ]:
# We will drop observations that are missing for this analysis

combined_data = combined_data.dropna()
combined_data.info()

In [ ]:
# In this section we will focus on the PE ratio.

final_dataframe = combined_data[['Ticker', 'Price', 'Price-to-Earnings Ratio']].copy(deep=True)
final_dataframe['Number of Shares to Buy'] = 'N/A'
final_dataframe

### b. Remove some growth stocks to narrow down our sample

In [ ]:
# We will consider dropping any negative PE ratio stocks and by keeping only the 15 highest PE ratios.

final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:15]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [ ]:
final_dataframe

### c. Calculate the number of shares to buy
We will create a function here so that we do not have to keep using this code to get the investment amount.

In [ ]:
def portfolio_input():
    global portfolio_size # allows us to use this value outside of the function
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        print(val)
    except ValueError:
        print('Please enter an integer value.')
        portfolio_size = input('How much money are you investing:')
        val = float(portfolio_size)
        print(val)

portfolio_input()

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe

### d. A more comprehensive approach
- The PE ratio (and every other valuation) has unique flaws
- For example, the PE ratio does not work well with negative values
- Stocks that buyback their own shares are difficult to value using the price-to-book ratio
- We will instead use a basket of valuation measures including:
    - Price-to-earnings
    - Price-to-book
    - Price-to-sales
    - Enterprise value over earnings before interest, tax, depreciation, and amortization
- Not all of these values are available so we must calculate some of them

In [ ]:
final_dataframe = combined_data.copy()
final_dataframe['Number of Shares to Buy'] = 'N/A'
final_dataframe

### e. Calculating ranks for each new measures
We must now rank each of these securities so that we can ultimately create a combined score that incorporates all four measures into one aggregate score.

In [ ]:
# agg_final will be a new dataframe that contains the measures for our aggregate scores

agg_final = final_dataframe.copy(deep=True)

metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV-to-EBITDA':'EV/EBITDA Percentile'
} # Label our new ranking variables with key:value pairs

for row in agg_final.index:
    for i in metrics.keys():
        # Loop through all of our measures (keys) and calculate the percentile ranking and save it as a new variable (values)
        agg_final.loc[row, metrics[i]] = stats.percentileofscore(agg_final[i], agg_final.loc[row, i])/100

#Print the entire DataFrame
agg_final

### f. Combining our scores into a new variable
We will now calculate our AGG_Score which is what we will use to filter stocks for our value investment strategy. AGG_Score will be a simple average of the four ranking values we just calculated.

In [ ]:
# Import a new library to calculate the mean values
from statistics import mean

for row in agg_final.index: # Iterate over each row in our datafarme
    value_ranks = [] # Create an empty list to store percentile rankings
    for i in metrics.keys():
        value_ranks.append(agg_final.loc[row, metrics[i]]) # Append each rows rankings to the empty list
    agg_final.loc[row, ' AGG_Score'] = mean(value_ranks) # Calculate the average of all rankings

agg_final

In [ ]:
# Retain only the lowest 20 possible stocks to invest in

agg_final.sort_values(by = ' AGG_Score', inplace = True)
agg_final = agg_final[:20]
agg_final.reset_index(drop = True, inplace = True)
agg_final

### g. Determine the number of shares to purchase if we equally weight our portfolio

In [ ]:
# Determine portfolio investment size

portfolio_input()

In [ ]:
position_size = float(portfolio_size) / len(agg_final.index)
position_size

In [ ]:
for i in range(0, len(agg_final)):
    agg_final.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / agg_final.loc[i, 'Price'])
agg_final

### h. Create a formatted output

In [ ]:
writer = pd.ExcelWriter('Value_Agg_trades.xlsx', engine='xlsxwriter')
agg_final.to_excel(writer, sheet_name='V Recommended Trades', index = False)

In [ ]:
# I would copy and paste this section rather than trying to learn all of the formatting options.
# This time we will make the number of shares to purchase a different color to draw attention to that column.

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format({'font_color': font_color,'bg_color': background_color,'border': 1})
dollar_template = writer.book.add_format({'num_format':'$0.00','font_color': font_color,'bg_color': background_color,'border': 1})
integer_template = writer.book.add_format({'num_format':'0','font_color': font_color,'bg_color': '#A9A9A9','border': 1})
float_template = writer.book.add_format({'num_format':'0.00','font_color': font_color,'bg_color': background_color,'border': 1})

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['EV-to-EBITDA', float_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['Price-to-Book Ratio', float_template],
                    'F': ['Price-to-Sales Ratio', float_template],
                    'G': ['Number of Shares to Buy', integer_template],
                    'H': ['PE Percentile', float_template],
                    'I': ['PB Percentile', float_template],
                    'J': ['PS Percentile', float_template],
                    'K': ['EV/EBITDA Percentile', float_template],
                    'L': ['AGG_SCORE', float_template],
                 }

for column in column_formats.keys():
    writer.sheets['V Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['V Recommended Trades'].write(f'{column}1', column_formats[column][0], string_template)

In [ ]:
# Is the output pretty? No, but you can play around with the formatting options to change that if you would like.
writer.save()